In [1]:
from module import connection
from module import get
from module import update
from module import search
from module import save
import pandas as pd

In [2]:
update.update_stock_info('000020_동화약품.csv')

20221105 20221117
20221106 20221117
20221107 20221117
20221108 20221117
20221109 20221117
20221110 20221117
20221111 20221117
20221112 20221117
20221113 20221117
20221114 20221117
20221115 20221117
20221116 20221117
20221117 20221117


,시간,시가,고가,저가,종가,전일대비,거래량,거래대금,누적체결매도수량,상장주식수,...,외국인현보유비율,수정주가일자,수정주가비율,기관순매수량,기관누적순매수량,등락주선,등락비율,예탁금,주식회전율,거래성립률
20221116,1530,9390,9390,9390,9390,0,1363,12800000,67745,0,...,0.0,0,0.0,0,0,0,0.0,0,0.0,0.0
20221116,1520,9400,9400,9390,9390,0,335,3150000,67745,0,...,0.0,0,0.0,0,0,0,0.0,0,0.0,0.0
20221116,1519,9390,9400,9390,9400,0,42,390000,67419,0,...,0.0,0,0.0,0,0,0,0.0,0,0.0,0.0
20221116,1518,9390,9400,9380,9400,0,52,490000,67391,0,...,0.0,0,0.0,0,0,0,0.0,0,0.0,0.0
20221116,1517,9390,9390,9380,9390,0,27,250000,67365,0,...,0.0,0,0.0,0,0,0,0.0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20201027,905,16200,16300,16100,16250,0,20521,332280000,62820,0,...,0.0,0,0.0,0,0,0,0.0,0,0.0,0.0
20201027,904,16400,16450,15900,16300,0,30445,493410000,50504,0,...,0.0,0,0.0,0,0,0,0.0,0,0.0,0.0
20201027,903,16500,16500,16250,16450,0,18800,307520000,26098,0,...,0.0,0,0.0,0,0,0,0.0,0,0.0,0.0
20201027,902,16650,16700,16300,16500,0,11727,194040000,17366,0,...,0.0,0,0.0,0,0,0,0.0,0,0.0,0.0
